# Set up google collab variables

In [1]:
collab = False

import os
import shutil
#libraries
from collections import OrderedDict
import json
import numpy as np

#visualization of the dataset
import matplotlib.pyplot as plt
import nibabel as nib

if collab:
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive', force_remount=True)
    # Change "neurosciences-segmentation" to the name of your project folder
    root_dir = "/content/drive/MyDrive/neurosciences-segmentation"


else:
    # get the dir of the parent dir
    root_dir = os.getcwd()


my_nnunet_dir = os.path.join(root_dir,'my_nnunet')
input_dir = os.path.join(root_dir, 'data/input')
segmentation_dir = os.path.join(root_dir, 'data/segmentation')
print(my_nnunet_dir)
print(input_dir)
print(segmentation_dir)

/Users/francoisporcher/Downloads/awesome-ai-tutorials/004 - nnU-Net/my_nnunet
/Users/francoisporcher/Downloads/awesome-ai-tutorials/004 - nnU-Net/data/input
/Users/francoisporcher/Downloads/awesome-ai-tutorials/004 - nnU-Net/data/segmentation


In [2]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input:
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder
    """
    if os.path.exists(folder_path):

        if not overwrite:
            print(f'{folder_path} exists.')
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")


In [4]:
os.chdir(root_dir)
make_if_dont_exist('my_nnunet', overwrite=False)
os.chdir('my_nnunet')
print(f"Current working directory: {os.getcwd()}")

my_nnunet exists.
Current working directory: /Users/francoisporcher/Downloads/004 - nnU-Net/my_nnunet


In [3]:
#Now we create the two folders where you can put your input images and corresponding segmentations
make_if_dont_exist(input_dir)
make_if_dont_exist(segmentation_dir)

/Users/francoisporcher/Downloads/awesome-ai-tutorials/004 - nnU-Net/data/input created!
/Users/francoisporcher/Downloads/awesome-ai-tutorials/004 - nnU-Net/data/segmentation created!


# Clone nnunet repository and install requirements

In [ ]:
!pip install nnunet

In [ ]:
os.getcwd()

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git
!git clone https://github.com/NVIDIA/apex

In [ ]:
my_nnunet_dir

In [ ]:
# repository dir is the path of the github folder
respository_dir = os.path.join(my_nnunet_dir,'nnUNet')
os.chdir(respository_dir)
!pip install -e
!pip install --upgrade git+https://github.com/nanohanno/hiddenlayer.git@bugfix/get_trace_graph#egg=hiddenlayer

In [ ]:
# return in the my_nnunet folder
os.chdir(my_nnunet_dir)
os.getcwd()

# Creation of all the required folders

In [5]:
task_name = 'Task001' #change here for different task name

# We define all the necessary paths
nnunet_dir = "nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data"
task_folder_name = os.path.join(nnunet_dir,task_name) 
train_image_dir = os.path.join(task_folder_name,'imagesTr') # path to training images
train_label_dir = os.path.join(task_folder_name,'labelsTr') # path to training labels
test_dir = os.path.join(task_folder_name,'imagesTs') # path to test images
main_dir = os.path.join(my_nnunet_dir,'nnUNet/nnunet') # path to main directory
trained_model_dir = os.path.join(main_dir, 'nnUNet_trained_models') # path to trained models

In [6]:
# Creation of all the folders
overwrite = False # Set this to True if you want to overwrite the folders
make_if_dont_exist(task_folder_name,overwrite = overwrite)
make_if_dont_exist(train_image_dir, overwrite = overwrite)
make_if_dont_exist(train_label_dir, overwrite = overwrite)
make_if_dont_exist(test_dir,overwrite= overwrite)
make_if_dont_exist(trained_model_dir, overwrite=overwrite)

nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001 exists.
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/imagesTr exists.
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/labelsTr exists.
nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/imagesTs exists.
/Users/francoisporcher/Downloads/004 - nnU-Net/my_nnunet/nnUNet/nnunet/nnUNet_trained_models exists.


# Setting the environment variables

In [7]:
os.environ['nnUNet_raw_data_base'] = os.path.join(main_dir,'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = os.path.join(main_dir,'preprocessed')
os.environ['RESULTS_FOLDER'] = trained_model_dir

# Move the files in the training and test

In [8]:
def copy_and_rename(old_location,old_file_name,new_location,new_filename,delete_original = False):
    shutil.copy(os.path.join(old_location,old_file_name),new_location)
    os.rename(os.path.join(new_location,old_file_name),os.path.join(new_location,new_filename))
    if delete_original:
        os.remove(os.path.join(old_location,old_file_name))

In [9]:
segmentation_dir

'/Users/francoisporcher/Downloads/004 - nnU-Net/data/ground_truth'

In [10]:
list_of_all_files = os.listdir(segmentation_dir)

In [12]:
list_of_all_files = os.listdir(segmentation_dir)
list_of_all_files = [file_name for file_name in list_of_all_files if file_name.endswith('.nii.gz')]


for file_name in list_of_all_files:
    copy_and_rename(input_dir,file_name,train_image_dir,file_name)
    copy_and_rename(segmentation_dir,file_name,train_label_dir,file_name)

In [13]:
def check_modality(filename):
    """
    check for the existence of modality
    return False if modality is not found else True
    """
    end = filename.find('.nii.gz')
    modality = filename[end-4:end]
    for mod in modality:
        if not(ord(mod)>=48 and ord(mod)<=57): #if not in 0 to 9 digits
            return False
    return True

In [14]:
def rename_for_single_modality(directory):

    for file in os.listdir(directory):

        if check_modality(file)==False:
            new_name = file[:file.find('.nii.gz')]+"_0000.nii.gz"
            os.rename(os.path.join(directory,file),os.path.join(directory,new_name))
            print(f"Renamed to {new_name}")
        else:
            print(f"Modality present: {file}")

rename_for_single_modality(train_image_dir)
# rename_for_single_modality(test_dir)


Renamed to MMRR-21-7_0000.nii.gz
Renamed to OASIS-TRT-20-1_0000.nii.gz
Renamed to NKI-RS-22-15_0000.nii.gz
Renamed to NKI-RS-22-19_0000.nii.gz
Renamed to Afterthought-1_0000.nii.gz
Renamed to MMRR-21-10_0000.nii.gz
Renamed to OASIS-TRT-20-20_0000.nii.gz
Renamed to HLN-12-2_0000.nii.gz
Renamed to NKI-TRT-20-14_0000.nii.gz
Renamed to NKI-TRT-20-1_0000.nii.gz
Renamed to NKI-RS-22-3_0000.nii.gz
Renamed to OASIS-TRT-20-16_0000.nii.gz
Renamed to NKI-TRT-20-18_0000.nii.gz
Renamed to NKI-TRT-20-20_0000.nii.gz
Renamed to MMRR-21-12_0000.nii.gz
Renamed to NKI-RS-22-1_0000.nii.gz
Renamed to OASIS-TRT-20-14_0000.nii.gz
Renamed to Twins-2-1_0000.nii.gz
Renamed to OASIS-TRT-20-18_0000.nii.gz
Renamed to HLN-12-11_0000.nii.gz
Renamed to NKI-TRT-20-16_0000.nii.gz
Renamed to NKI-TRT-20-3_0000.nii.gz
Renamed to MMRR-21-9_0000.nii.gz
Renamed to NKI-RS-22-21_0000.nii.gz
Renamed to OASIS-TRT-20-3_0000.nii.gz
Renamed to MMRR-21-5_0000.nii.gz
Renamed to NKI-RS-22-17_0000.nii.gz
Renamed to NKI-TRT-20-12_0000.n

In [ ]:
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:

    json_dict = OrderedDict()
    json_dict['name'] = task_name
    json_dict['description'] = "Segmentation of T1 Scans from MindBoggle"
    json_dict['tensorImageSize'] = "3D"
    json_dict['reference'] = "see challenge website"
    json_dict['licence'] = "see challenge website"
    json_dict['release'] = "0.0"

    #you may mention more than one modality
    json_dict['modality'] = {
        "0": "MRI"
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "0": "Non Brain",
        "1": "Cortical gray matter",
        "2": "Cortical White matter",
        "3" : "Cerebellum gray ",
        "4" : "Cerebellum white"
    }


    train_ids = os.listdir(train_label_dir)
    test_ids = os.listdir(test_dir)
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTest'] = len(test_ids)

    #no modality in train image and labels in dataset.json
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    #removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % (i[:i.find("_0000")]+'.nii.gz') for i in test_ids]

    with open(os.path.join(task_folder_name,"dataset.json"), 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
        if json_file_exist==False:
            print('dataset.json created!')
        else:
            print('dataset.json overwritten!')

In [ ]:
!nnUNet_plan_and_preprocess -t 1 --verify_dataset_integrity

# Training the 3 models

In [ ]:
#train 3D full resolution U net
!nnUNet_train 3d_fullres nnUNetTrainerV2 1 0 --npz 

In [ ]:
# train 2D U net
!nnUNet_train 2d nnUNetTrainerV2 1 0 --npz

In [ ]:
# train 3D U-net cascade
!nnUNet_train 3d_lowres nnUNetTrainerV2CascadeFullRes 1 0 --npz
!nnUNet_train 3d_fullres nnUNetTrainerV2CascadeFullRes 1 0 --npz

# Identifying the best configuration

In [ ]:
!nnUNet_find_best_configuration -m 2d 3d_fullres 3d_lowres 3d_cascade_fullres -t 1

# Inference

In [ ]:
os.getcwd()

In [ ]:
task_folder_name

In [ ]:
result_dir = os.path.join(task_folder_name, 'nnUNet_Prediction_Results')
make_if_dont_exist(result_dir, overwrite=True)

In [ ]:
!nnUNet_predict -i /content/drive/MyDrive/neurosciences-segmentation/my_nnunet/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/imagesTs -o /content/drive/MyDrive/neurosciences-segmentation/my_nnunet/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001/nnUNet_Prediction_Results -t 1 -tr nnUNetTrainerV2 -m 2d -f 0  --num_threads_preprocessing 1